# Save extended QC for bwm dataset

In [5]:
prefix = '/home/ines/repositories/'
# prefix = '/Users/ineslaranjeira/Documents/Repositories/'

In [3]:
"""
Check sessions available with dlc data
@author: Ines
"""
import pandas as pd
import pickle 
import os
import numpy as np
from one.api import ONE
import matplotlib.pyplot as plt
from brainbox.io.one import SessionLoader
from dateutil import parser

# Get my functions
functions_path =  prefix + '/representation_learning_variability/Functions/'

os.chdir(functions_path)
from video_functions import get_lick_times, lick_psth
# from data_processing import save_and_log

one = ONE(mode='remote')


# Lightning pose QC

In [7]:
ext_qc_path = prefix + 'representation_learning_variability/Video and wheel/Video QC/'
os.chdir(ext_qc_path)
ext_qc = pickle.load(open(ext_qc_path + "extended_qc07-10-2025", "rb"))  # This is the one from the paper code
use_qc = ext_qc.loc[ext_qc['task'].isin(['PASS', 'WARNING'])]


FileNotFoundError: [Errno 2] No such file or directory: '/home/ines/repositories/representation_learning_variability/Video and wheel/Video QC/extended_qc07-10-2025'

In [4]:
final_qc = use_qc.loc[(use_qc['_lightningPoseLeft_lick_detection'].isin(['PASS'])) &
                             (use_qc['_lightningPoseLeft_time_trace_length_match'].isin(['PASS'])) &   
                             (use_qc['_videoLeft_pin_state'].apply(lambda x: (isinstance(x, list) and True in x) or x == 'PASS')) &
                             (use_qc['_lightningPoseLeft_trace_all_nan'].isin(['PASS'])) &
                             (use_qc['_videoLeft_framerate'].apply(lambda x: (isinstance(x, list) and True in x) or x == 'PASS')) &   
                             (use_qc['_videoLeft_camera_times'].apply(lambda x: (isinstance(x, list) and True in x) or x == 'PASS')) &   
                             (use_qc['_videoLeft_dropped_frames'].apply(lambda x: (isinstance(x, list) and True in x) or x == 'PASS')) &  # can make more conservative by removing or  x == None
                             (use_qc['_videoLeft_timestamps'].isin([True, 'PASS']))                   
                            ]

print(len(final_qc))

218


In [8]:
# Load preprocessed data
results_path = prefix + 'representation_learning_variability/paper-individuality/'
filename = str(results_path + '1_bwm_qc_07-10-2025')
bwm_query = pickle.load(open(filename, "rb"))

/tmp/ipykernel_27244/2377349551.py:4: DeprecationWarning: numpy.core.numeric is deprecated and has been renamed to numpy._core.numeric. The numpy._core namespace contains private NumPy internals and its use is discouraged, as NumPy internals can change without warning in any release. In practice, most real-world usage of numpy.core is to access functionality in the public NumPy API. If that is the case, use the public NumPy API. If not, you are using NumPy internals. If you would still like to access an internal attribute, use numpy._core.numeric._frombuffer.
  bwm_query = pickle.load(open(filename, "rb"))


In [13]:
def get_XYs(one, eid, view, likelihood_thresh=0.9, lp=True):
    try:
        times = one.load_dataset(eid, '_ibl_%sCamera.times.npy' % view)
        if lp:
            cam = one.load_dataset(eid, '_ibl_%sCamera.lightningPose.pqt' % view)
        else:
            cam = one.load_dataset(eid, '_ibl_%sCamera.dlc.pqt' % view)
    except KeyError:
        print('not all data available')
        return None, None

    if lp:
        keys = cam.keys()
        sufixes = np.array(['_'.join(x.split('_')[-1:]) for x in keys])
        Xs = keys[np.where(sufixes == 'x')]
        points = np.array(['_'.join(x.split('_')[:-1]) for x in Xs])
    else:
        points = np.unique(['_'.join(x.split('_')[:-1]) for x in cam.keys()])

    # Set values to nan if likelyhood is too low # for pqt: .to_numpy()
    XYs = {}
    for point in points:
        x = np.ma.masked_where(cam[point + '_likelihood'] < likelihood_thresh, cam[point + '_x'])
        x = x.filled(np.nan)
        y = np.ma.masked_where(cam[point + '_likelihood'] < likelihood_thresh, cam[point + '_y'])
        y = y.filled(np.nan)
        XYs[point] = np.array([x, y]).T
    return times, XYs


""" LICKs """

def get_licks(XYs):

    '''
    define a frame as a lick frame if
    x or y for left or right tongue point
    change more than half the sdt of the diff
    '''  
    
    licks = []
    for point in ['tongue_end_l', 'tongue_end_r']:
        for c in XYs[point].T:
           thr = np.nanstd(np.diff(c))/4
           licks.append(set(np.where(abs(np.diff(c))>thr)[0]))
    return sorted(list(set.union(*licks))) 


def get_lick_times(one, eid, lp, combine=False, video_type='left'):
    
    if combine:    
        # combine licking events from left and right cam
        lick_times = []
        for video_type in ['right','left']:
            times, XYs = get_XYs(one, eid, video_type, likelihood_thresh=0.9, lp=lp)
            r = get_licks(XYs)
            # cover case that there are less times than DLC points            
            idx = np.where(np.array(r)<len(times))[0][-1]            
            lick_times.append(times[r[:idx]])
        
        # lick_times = sorted(np.concatenate(lick_times))
        lick_times = np.array(sorted(np.concatenate(lick_times)))
        
    else:
        times, XYs = get_XYs(one, eid, video_type, likelihood_thresh=0.9, lp=lp)    
        r = get_licks(XYs)
        # cover case that there are less times than DLC points
        idx = np.where(np.array(r)<len(times))[0][-1]              
        lick_times = times[r[:idx]]
        # lick_times = times[r]

    return lick_times


def get_feature_event_times(dlc, dlc_t, features):
    """
    Detect events from the dlc traces. Based on the standard deviation between frames
    :param dlc: dlc pqt table
    :param dlc_t: dlc times
    :param features: features to consider
    :return:
    """

    for i, feat in enumerate(features):
        f = dlc[feat]
        threshold = np.nanstd(np.diff(f)) / 4
        if i == 0:
            events = np.where(np.abs(np.diff(f)) > threshold)[0]
        else:
            events = np.r_[events, np.where(np.abs(np.diff(f)) > threshold)[0]]

    return dlc_t[np.unique(events)]

## Plot lick PSTHs

In [ ]:
# Alternative loading
 
# sl = SessionLoader(eid=eid, one=one)
# sl.load_pose(views=['left'])
# poses = sl.pose['leftCamera']

# dlc = sl.pose['leftCamera']
# dlc_t = sl.pose['leftCamera']['times']
# features = ['tongue_end_l_x', 'tongue_end_l_y','tongue_end_r_x', 'tongue_end_r_y']
# lick_times = get_feature_event_times(dlc, dlc_t, features)
# len(lick_times)

/opt/anaconda3/envs/iblenv/lib/python3.10/site-packages/brainbox/behavior/dlc.py:59: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  dlc[f'{feat}_x'][nan_fill] = np.nan
/opt/anaconda3/envs/iblenv/lib/python3.10/site-packages/brainbox/behavior/

In [19]:
session = bwm_query['eid'].unique()[10]
sl = SessionLoader(eid=session, one=one)
sl.load_pose(views=['left'])
poses = sl.pose['leftCamera']

dlc = sl.pose['leftCamera']
dlc_t = sl.pose['leftCamera']['times']
features = ['tongue_end_l_x', 'tongue_end_l_y','tongue_end_r_x', 'tongue_end_r_y']
lick_times = get_feature_event_times(dlc, dlc_t, features)

/home/ines/miniconda3/envs/iblenv/lib/python3.9/site-packages/one/util.py:543: ALFWarning: Multiple revisions: "", "2025-06-18", "2025-03-03", "2025-06-01", "2025-05-29", "2024-05-06", "2025-05-31"
  warnings.warn(f'Multiple revisions: {rev_list}', alferr.ALFWarning)


In [5]:
save_path = prefix + '/representation_learning_variability/Video and wheel/Video QC/Figures/2_cameras/'
files = os.listdir(save_path)
counter = 0
eids = final_qc['eid'].unique()
for eid in eids:

    filename = str(eid + '_2_camera_lick_psth_-12_feedback.png')
    if filename not in files:
        counter = counter +1

In [6]:
print(counter)

22


In [ ]:
save_path = prefix + '/representation_learning_variability/Video and wheel/Video QC/Figures/2_cameras/'
files = os.listdir(save_path)

eids = final_qc['eid'].unique()
for eid in eids:

    filename = str(eid + '_2_camera_lick_psth_-12_feedback.png')
    if filename not in files:

        fig, ax = plt.subplots(nrows=2, ncols=1, sharex=True, sharey=True, figsize=[7, 7])
        trials = one.load_object(eid, obj='trials', namespace='ibl')
        trial_df = trials.to_df()

        try:
            lick_times_lp = get_lick_times(one, eid, lp=True, combine=True, video_type = 'left')
            data_lp = lick_psth(trial_df, lick_times_lp, 1, 2, event='feedback_times')

            lick_times_dlc = get_lick_times(one, eid, lp=False, combine=True, video_type = 'left')
            data_dlc = lick_psth(trial_df, lick_times_dlc, 1, 2, event='feedback_times')

            for d, data in enumerate([data_dlc, data_lp]):
                trials = data['trial']
                num_trials = int(np.max(trials))
                # Plot data
                for t, trial in enumerate(range(num_trials)):
                    licks_correct = np.array(data.loc[(data['trial']==t) & (data['correct']==1), 'lick_times'])
                    licks_incorrect = np.array(data.loc[(data['trial']==t) & (data['correct']==-1), 'lick_times'])
                    ax[d].scatter(licks_correct, np.full_like(licks_correct, t), color='green', s=1)
                    ax[d].scatter(licks_incorrect, np.full_like(licks_incorrect, t), color='red', s=1)
                ax[1].set_xlabel('Time from feedback')
                ax[d].set_ylabel('Trial')
                if d == 0:
                    ax[d].set_title('DLC - ' +eid)
                else:
                    ax[d].set_title('LP - ' +eid)
            plt.tight_layout()
            plt.savefig(save_path+filename, format='png')
            # plt.show()
        except:
            print('Missing data for mouse '+eid)


/opt/anaconda3/envs/iblenv/lib/python3.10/site-packages/one/api.py:291: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '[nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan 

Downloading: /Users/ineslaranjeira/Downloads/FlatIron/angelakilab/Subjects/NYU-30/2020-10-22/001/alf/_ibl_rightCamera.lightningPose.e30c2d51-bdc1-4a9d-9f97-35f0bba5ca23.pqt Bytes: 415277062


100%|██████████| 396.0390682220459/396.0390682220459 [03:16<00:00,  2.01it/s]
/Users/ineslaranjeira/Documents/Repositories/representation_learning_variability/Functions/video_functions.py:172: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  licks_df = pd.concat([licks_df, temp_df])
/opt/anaconda3/envs/iblenv/lib/python3.10/site-packages/numpy/lib/nanfunctions.py:1872: RuntimeWarning: Degrees of freedom <= 0 for slice.
  var = nanvar(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
/opt/anaconda3/envs/iblenv/lib/python3.10/site-packages/numpy/lib/nanfunctions.py:1872: RuntimeWarning: Degrees of freedom <= 0 for slice.
  var = nanvar(a, axis=axis, dtype=dtype, out=out, ddof=ddof,


Missing data for mouse 5ec72172-3901-4771-8777-6e9490ca51fc


local md5 mismatch on dataset: danlab/Subjects/DY_016/2020-09-13/001/alf/_ibl_rightCamera.times.npy


...

100%|██████████| 0.06616020202636719/0.06616020202636719 [00:00<00:00,  4.99s/it]
No default revision for dataset alf/probe00/pykilosort/clusters.amps.npy; using most recent


Downloading: /Users/ineslaranjeira/Downloads/FlatIron/zadorlab/Subjects/CSH_ZAD_026/2020-09-04/001/alf/_ibl_rightCamera.times.b524fe5d-7483-4af2-970b-bc07fbcec66f.npy Bytes: 7408856


100%|██████████| 7.065635681152344/7.065635681152344 [00:03<00:00,  2.06it/s]
No default revision for dataset alf/probe00/pykilosort/clusters.amps.npy; using most recent


Downloading: /Users/ineslaranjeira/Downloads/FlatIron/zadorlab/Subjects/CSH_ZAD_026/2020-09-04/001/alf/_ibl_rightCamera.lightningPose.e81c0281-6aaf-4630-b3d4-69915cde1144.pqt Bytes: 414216502


 17%|█▋        | 68.0/395.0276393890381 [00:31<02:46,  1.96it/s]